In [1]:
from elasticsearch import Elasticsearch

In [13]:
ELASTIC_USERNAME = "team4" # INSERT YOUR USERNAME
ELASTIC_PASSWORD = "jBs1QvrvDF6wH" # INSERT YOUR PASSWORD


hosts = ["https://eestec2024.cl-test.medius.si:9200"]
# How many samples the query returns (maximum is 10000) if you want more use scroll
number_of_samples = 1000

## Initialize client

In [3]:
# Connect to the Elasticsearch cluster
es = Elasticsearch(
    hosts,
    ssl_show_warn=False,  # Suppress SSL warnings, use with caution
    verify_certs=False,  # This skips SSL certificate verification
    http_auth=(ELASTIC_USERNAME, ELASTIC_PASSWORD)  # Include if your Elasticsearch has authentication enabled
)

C:\Users\grabn\AppData\Local\Temp\ipykernel_30544\1703270324.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


## Get a list of available indices
This elastic cluster is constructed so that all the data from each day is contained in its own index.

In [4]:
# Print a list of all indices
indices = es.indices.get_alias(index="geo-locations-*")  # Using `index` as a keyword argument
for index in indices:
    print(index)

geo-locations-12.01.2023
geo-locations-10.01.2023
geo-locations-09.01.2023
geo-locations-11.01.2023


## Without scroll
This way of obtaining data returns only the number of samples specified (the maximum here is 10000)

In [26]:
# Define your search query
# search_query = {
#     "sort" : [
#       {"dateTimeEvent" : {"order" : "desc", "mode" : "min"}}
#    ],
#     "query": {
#         "range": {
#             "dateTimeEvent": {
#                 "gte": "10.01.2023 00:00:02",
#                 "lt": "10.01.2023 23:59:03"
#             }
#         }
#     },
#     "_source": ["dateTimeEvent", "msisdn","location"],  # Specify the fields to return
#     "size": number_of_samples  # Specify the number of documents to return
# }

search_query = {
    "sort" : [
      {"dateTimeEvent" : {"order" : "desc", "mode" : "min"}}
   ],
    "query": {
        "bool": {
            "must": {
                "range": {
                    "dateTimeEvent": {
                        "gte": "10.01.2023 09:09:02",
                        "lt": "10.01.2023 23:59:03"
                    }
                }
            },
            "filter": {
                "geo_distance": {
                    "distance": "4km",
                    "location": {
                        "lat": 46.04980,
                        "lon": 14.51522
                    }
                }
            }
        }
    },
    "_source": ["dateTimeEvent", "msisdn","location"],
    "size": number_of_samples
}

# Execute the search query
response = es.search(index="geo-locations-10.01.2023", body=search_query)

# Print the search results
for hit in response['hits']['hits']:
    print(hit['_source'])

{'dateTimeEvent': '10.01.2023 23:59:02', 'msisdn': 'c91374e4430deff00456c0415d85f6dbf992391fd40c687829ad99e4e17c6640', 'location': {'lon': 14.54464, 'lat': 46.05538}}
{'dateTimeEvent': '10.01.2023 23:59:02', 'msisdn': 'a48f9d0e7a3966096ec25066a0a8c330a8cfdf252fbcbfae974e8c71d1fcf8e0', 'location': {'lon': 14.50223, 'lat': 46.04278}}
{'dateTimeEvent': '10.01.2023 23:59:02', 'msisdn': '37da957f80f8aa04eb00088b035159bc22819396f39eb3b7310697672e323e0c', 'location': {'lon': 14.50393, 'lat': 46.05548}}
{'dateTimeEvent': '10.01.2023 23:59:02', 'msisdn': '355114d3248ed670a10980a9de33a9f9c967a191d15df80c126978af31319fb0', 'location': {'lon': 14.50665, 'lat': 46.05584}}
{'dateTimeEvent': '10.01.2023 23:59:02', 'msisdn': '5d5f6c8ac23a6083c234ee84f1c6abc6d6038a00cbda1917113aeafdfc6a6164', 'location': {'lon': 14.50777, 'lat': 46.06167}}
{'dateTimeEvent': '10.01.2023 23:59:02', 'msisdn': '20722680a40e6a679198657ccb2fa9d24d26518a5d3705988cd8bb6e6619d6f6', 'location': {'lon': 14.53708, 'lat': 46.02214}

## With scroll
This way of pbtaining data returns the number of samples specified and then makes anothe request to the elasticsearch untill all data is consumed.

In [ ]:
# Define your search query
search_query = {
    "query": {
        "match_all": {}  # This is a simple query that matches all documents
    },
    "size": number_of_samples
}

# Execute the search query
response = es.search(index="<INSERT_INDEX_NAME>",
                     body=search_query,
                     scroll='1m'  # Keep the scroll window open for 1 minute
)

# Get the scroll ID
scroll_id = response['_scroll_id']

# Keep scrolling while we have data
while len(response['hits']['hits']):
    # Process hits here
    print("Received:", len(response['hits']['hits']), "hits")

    # Make a request using the Scroll API
    response = es.scroll(
        scroll_id=scroll_id,
        scroll='1m'  # Extend the scroll window for another minute
    )

    # Update the scroll ID
    scroll_id = response['_scroll_id']

# Clear the scroll when you're done
es.clear_scroll(scroll_id=scroll_id)